<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Generate-vocabularies" data-toc-modified-id="Generate-vocabularies-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Generate vocabularies</a></span></li><li><span><a href="#Save-dictionaries-to-file" data-toc-modified-id="Save-dictionaries-to-file-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Save dictionaries to file</a></span></li><li><span><a href="#Tokenzize/pad-inputs-and-labels,-encode-labels" data-toc-modified-id="Tokenzize/pad-inputs-and-labels,-encode-labels-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Tokenzize/pad inputs and labels, encode labels</a></span></li><li><span><a href="#Split-inputs-and-labels-data" data-toc-modified-id="Split-inputs-and-labels-data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Split inputs and labels data</a></span></li><li><span><a href="#Define-and-train-model" data-toc-modified-id="Define-and-train-model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Define and train model</a></span></li></ul></div>

In [1]:
import json, keras, numpy as np
from collections import Counter
from sklearn.metrics import confusion_matrix, classification_report
from keras.optimizers import Adam
from keras.utils import plot_model
from keras.utils import to_categorical
from keras.models import Model, load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Embedding, Dense, Input, Flatten, LSTM, Bidirectional, TimeDistributed, Activation
from numpy.random import seed
from tensorflow import set_random_seed
seed(1)
set_random_seed(1)

Using TensorFlow backend.


In [2]:
# Load dataset
processed_folder = '../data/processed/'
# train_inputs = open('../data/train_inputs.txt').readlines()
# # train_labels = open('../data/train_labels.txt').readlines()
# dev_inputs = open(processed_folder+'dev_inputs.txt').readlines()
# dev_labels = open(processed_folder+'dev_labels.txt').readlines()
test_inputs = open(processed_folder+'test_inputs.txt').readlines()
test_labels = open(processed_folder+'test_labels.txt').readlines()

In [3]:
# print('Number of training sentences: {}, Number of label sentences: {}'.format(len(train_inputs), \
#       len(train_labels)))
# print('Number of dev sentences: {}, Number of label sentences: {}'.format(len(dev_inputs), \
#       len(dev_labels)))
print('Number of input sentences: {}, Number of label sentences: {}'.format(len(test_inputs), \
      len(test_labels)))

Number of input sentences: 30000, Number of label sentences: 30000


### Generate vocabularies

In [4]:
inputs_data = ''.join(test_inputs)
char_in_inputs = Counter(inputs_data)
inputs_vocab = sorted(char_in_inputs, key=char_in_inputs.get, reverse=True)
inputs_vocab_int = {char:idx for idx, char in enumerate(inputs_vocab, 2)}
inputs_vocab_int['pad'] = 0
inputs_vocab_int['unk'] = 1

labels_data = ''.join(test_labels)
char_in_labels = Counter(labels_data)
labels_vocab = sorted(char_in_labels, key=char_in_labels.get, reverse=True)
labels_vocab_int = {char:idx for idx, char in enumerate(labels_vocab, 2)}
labels_vocab_int['pad'] = 0
labels_vocab_int['unk'] = 1

num_inputs_vocab = len(inputs_vocab_int)
num_labels_vocab = len(labels_vocab_int)

print('Inputs vocab: {}, Labels vocab: {}'.format(num_inputs_vocab, num_labels_vocab))

Inputs vocab: 1126, Labels vocab: 69


### Save dictionaries to file

In [5]:
with open('../data/processed/inputs_vocabs.json', 'w') as f:
        json.dump(inputs_vocab_int, f, indent=4)
        
with open('../data/processed/labels_vocabs.json', 'w') as f:
        json.dump(labels_vocab_int, f, indent=4)

### Tokenzize/pad inputs and labels, encode labels

In [6]:
max_seq_len = 256
inputs_tokenize = []

for sentence in test_inputs:
    inputs_tokenize.append([inputs_vocab_int[char] for char in sentence])
    
inputs_tokenize_pad = pad_sequences(sequences=inputs_tokenize, maxlen=max_seq_len, padding='post', value=0)

labels_tokenize = []
for sentence in test_labels:
    labels_tokenize.append([labels_vocab_int[char] for char in sentence])
    
labels_tokenize_pad = pad_sequences(sequences=labels_tokenize, maxlen=max_seq_len, padding='post', value=0)

encode_labels = to_categorical(y=labels_tokenize_pad, num_classes=len(labels_vocab_int))

### Split inputs and labels data

In [7]:
model_inputs, model_labels = inputs_tokenize_pad[:int(-(0.2*len(inputs_tokenize_pad)))], \
                                encode_labels[:int(-(0.2*len(encode_labels)))]
model_test_inputs, model_test_labels = inputs_tokenize_pad[int(-(0.2*len(inputs_tokenize_pad))):], \
                                        encode_labels[int(-(0.2*len(encode_labels))):]

In [8]:
model_inputs.shape

(24000, 256)

In [9]:
model_labels.shape

(24000, 256, 69)

### Define and train model

In [7]:
chkpt = ModelCheckpoint(filepath='../models/model.h5', monitor='val_loss', verbose=1, save_best_only=True)
embed_input = Input(shape=(max_seq_len,))
embedding = Embedding(input_dim=num_inputs_vocab, output_dim=128, input_length=max_seq_len, \
                      trainable=True)(embed_input)
blstm = Bidirectional(LSTM(units=128, return_sequences=True))(embedding)
output = TimeDistributed(Dense(units=num_labels_vocab, activation='softmax'))(blstm)
model = Model(inputs=[embed_input], outputs=[output])
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), 
              metrics=['accuracy'])
model.summary()

W0729 17:55:18.688277 4566152640 deprecation_wrapper.py:119] From /Users/leotay/Applications/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0729 17:55:18.689251 4566152640 deprecation_wrapper.py:119] From /Users/leotay/Applications/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0729 17:55:18.692223 4566152640 deprecation_wrapper.py:119] From /Users/leotay/Applications/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0729 17:55:19.094277 4566152640 deprecation_wrapper.py:119] From /Users/leotay/Applications/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 256, 128)          146944    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256, 256)          263168    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 256, 47)           12079     
Total params: 422,191
Trainable params: 422,191
Non-trainable params: 0
_________________________________________________________________


In [8]:
model = model.fit(x=model_inputs, y=model_labels, batch_size=128, epochs=1, shuffle=True, \
                 validation_split=0.2, verbose=1, callbacks=[chkpt])

W0729 17:55:19.197869 4566152640 deprecation.py:323] From /Users/leotay/Applications/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0729 17:55:20.103088 4566152640 deprecation_wrapper.py:119] From /Users/leotay/Applications/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 19200 samples, validate on 4800 samples
Epoch 1/1
19200/19200 [==============================] - 117s 6ms/step - loss: 0.7200 - acc: 0.9004 - val_loss: 0.4275 - val_acc: 0.9276

Epoch 00001: val_loss improved from inf to 0.42749, saving model to ../models/model.h5


In [9]:
model = load_model('../models/model.h5')
plot_model(model=model, to_file='./model.png', show_shapes=True)

In [10]:
cm_pred = model.predict(model_test_inputs)
out_pred = [np.argmax(x, axis=1) for x in cm_pred]
out_pred = np.concatenate(out_pred, axis=0)
y_ = [np.argmax(y, axis=1) for y in model_test_labels]
y_ = np.concatenate(y_, axis=0)

In [11]:
con_matrix = confusion_matrix(y_true=y_, y_pred=out_pred)

In [12]:
con_matrix

array([[772837,   1849,      0, ...,      0,      0,      0],
       [  5003, 649726,      0, ...,      0,      0,      0],
       [     9,  12235,      0, ...,      0,      0,      0],
       ...,
       [     0,    108,      0, ...,      0,      0,      0],
       [     0,     96,      0, ...,      0,      0,      0],
       [     0,     22,      0, ...,      0,      0,      0]])

In [13]:
class_rept = classification_report(y_true=y_, y_pred=out_pred, output_dict=True)

/Users/leotay/Applications/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [14]:
class_rept

{'0': {'precision': 0.9901870092864025,
  'recall': 0.9976132265201643,
  'f1-score': 0.9938862461113875,
  'support': 774686},
 '2': {'precision': 0.8600140572322607,
  'recall': 0.992331389070893,
  'f1-score': 0.9214468561865795,
  'support': 654747},
 '3': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 12244},
 '4': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 9890},
 '5': {'precision': 0.14285714285714285,
  'recall': 0.0005,
  'f1-score': 0.000996512207274539,
  'support': 6000},
 '6': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 4616},
 '7': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 4498},
 '8': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 4503},
 '9': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 3736},
 '10': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 4324},
 '11': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 4609},
 '12': {'precision': 0.0,